# Notebook A: Experimental Growth Parameter Calculations

This Jupyter notebook is dedicated to the analysis and calculation of growth parameters for the olegenous yeast, <i>Y. lipolytica</i>, when grown with glucose, glycerol and oleic acid. 

### Objective
The objective of this analysis is to provide a clear and quantifiable understanding of the how of the growth rates, yield coefficients, and substrate uptake rates were calculated. This is crucial for providing constraints for the later genome scale model analysis.

### Important Functions
`get_average_growth_parameters`
This function computes the average growth parameters across multiple trials. This function leverages the get_trial_growth_parameters function for each trial and then averages the results.

`get_trial_growth_parameters`
This function calculates growth parameters for a single trial. It computes the growth rate, yield coefficient, and substrate uptake rate for a given trial number.

### A note about oleic acid yield
Due to difficulties in measuring the partially insoluable oleic acid, the oleic acid yield was determined based on the yields of glucose and glycerol. The process involved normalizing glucose and glycerol yields to a single carbon basis, averaging these values, and extrapolating to oleic acid's 18 carbons. The standard deviation was calculated in the same way

### Load imports

In [ ]:
import pandas as pd
import sys

source_dir = '../src'
sys.path.append(source_dir)

from growth_parameters.get_average_growth_parameters import get_average_growth_parameters

### Load glucose growth curve data

In [ ]:
# load glucose growth data
glucose_growth_df = pd.read_csv('../data/growth_curves/glucose_growth_data.csv')

glucose_growth_df

### Get glucose growth parameters

In [ ]:
glucose_growth_parameters = get_average_growth_parameters(growth_df=glucose_growth_df, substrate='glucose', molar_mass=180.16)

### Load glycerol data

In [ ]:
# load glycerol growth data
glycerol_growth_df = pd.read_csv('../data/growth_curves/glycerol_growth_data.csv')

glycerol_growth_df

### Calculate glycerol parameters

In [ ]:
glycerol_growth_parameters = get_average_growth_parameters(growth_df=glycerol_growth_df, substrate='glycerol', molar_mass=92.09)

### Load oleic acid growth curve data

In [ ]:
# load oleic acid growth data
oleic_acid_growth_df = pd.read_csv('../data/growth_curves/oleic_acid_growth_data.csv')

oleic_acid_growth_df

### Get oleic acid growth parameters

In [ ]:
oleic_acid_growth_parameters = get_average_growth_parameters(growth_df=oleic_acid_growth_df, substrate='oleic_acid', molar_mass=282.46)

### Extrapolate glucose and glycerol yields to oleic acid

In [ ]:
# get the average yield coefficient normalized to number by carbon atoms (based on yield coefficients of glucose and glycerol)
glucose_yield_coefficient_per_c = glucose_growth_parameters['yield_coefficient'] / 6
glycerol_yield_coefficient_per_c = glycerol_growth_parameters['yield_coefficient'] / 3
average_yield_coefficient_per_c = (glucose_yield_coefficient_per_c + glycerol_yield_coefficient_per_c) / 2

# Get the standard deviation of  oleic acid yield coefficient normalized to number by carbon atoms 
glucose_yield_coefficient_per_c_std = glucose_growth_parameters['yield_coefficient_std'] / 6
glycerol_yield_coefficient_per_c_std = glycerol_growth_parameters['yield_coefficient_std'] / 3

# Calculating pooled standard deviation
average_yield_coefficient_per_c_std = (glucose_yield_coefficient_per_c_std**2 + glycerol_yield_coefficient_per_c_std**2)**0.5 / 2

# extrapolate the average yield coefficient to oleic acid (18 carbons)
oleic_acid_yield_coefficient = average_yield_coefficient_per_c * 18
oleic_acid_yield_coefficient_std = average_yield_coefficient_per_c_std * 18

# add the oleic acid yield coefficient to the growth parameters dictionary
oleic_acid_growth_parameters['yield_coefficient'] = oleic_acid_yield_coefficient
oleic_acid_growth_parameters['yield_coefficient_std'] = oleic_acid_yield_coefficient_std

print(f'oleic acid yield coefficient = {oleic_acid_yield_coefficient:.3f} ± {oleic_acid_yield_coefficient_std:.3f} g biomass/mmol oleic acid')

### Calculate oleic acid substrate uptake rate

In [ ]:
# calculate the substrate uptake rate
substrate_uptake_rate = oleic_acid_growth_parameters['growth_rate'] / oleic_acid_growth_parameters['yield_coefficient']

# calculate the standard deviation of the substrate uptake rate
partial_mu = 1 / oleic_acid_growth_parameters['yield_coefficient']
partial_Y = -oleic_acid_growth_parameters['growth_rate'] / (oleic_acid_growth_parameters['yield_coefficient'] ** 2)
substrate_uptake_rate_std = ((partial_mu * oleic_acid_growth_parameters['growth_rate_std']) ** 2 + (partial_Y * oleic_acid_growth_parameters['yield_coefficient_std']) ** 2) ** 0.5

# add the substrate uptake rate to the growth parameters dictionary
oleic_acid_growth_parameters['substrate_uptake_rate'] = substrate_uptake_rate
oleic_acid_growth_parameters['substrate_uptake_rate_std'] = substrate_uptake_rate_std

print(f'oleic acid substrate uptake rate = {substrate_uptake_rate:.3f} ± {substrate_uptake_rate_std:.3f} mmol oleic acid/gram biomass * hr')

### Save measured biomass yields as a csv

In [ ]:
# generate a dataframe of the average growth parameters
average_growth_parameters_df = pd.DataFrame({
    'glucose': glucose_growth_parameters,
    'glycerol': glycerol_growth_parameters,
    'oleic_acid': oleic_acid_growth_parameters
})

# save as a csv
average_growth_parameters_df.to_csv('../results/growth_parameters/growth_parameters.csv')

average_growth_parameters_df
